# Main Notebook for Leakage Detection

## Setup

### Google Colab Setup

In [1]:
# Mount Google Colab files

from google.colab import drive
drive.mount('/content/gdrive')

%cd /content/gdrive/MyDrive/Bachelorarbeit/BA-WDS-Leakage-Detection

Mounted at /content/gdrive
/content/gdrive/MyDrive/Bachelorarbeit/BA-WDS-Leakage-Detection


In [ ]:
import utils

import importlib
importlib.reload(utils)

### Imports

In [ ]:
!pip install wntr
%matplotlib inline
import wntr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Main

In [ ]:
print('Hello World')